In [332]:
import sys
print(sys.version)
%load_ext autoreload
%autoreload 0


from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from spread import get_spread_data

SALARY_FILE = './data/fanduel_9_20.csv'
WEEK = 3
ACTIVE_FILE = './data/active.csv'

MIN_FAVORED = 3
print('ready')

3.7.11 (default, Jul  6 2021, 12:43:19) 
[Clang 12.0.5 (clang-1205.0.22.9)]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [333]:
spread_df = get_spread_data(WEEK)
spread_df.info()

return cached data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 44 columns):
Unnamed: 0                      16 non-null int64
SeasonType                      16 non-null int64
Season                          16 non-null int64
Type                            16 non-null object
Week                            16 non-null int64
Timeframe                       16 non-null object
Day                             16 non-null object
DateTime                        16 non-null object
DayString                       16 non-null object
TimeString                      16 non-null object
DayOfWeek                       16 non-null object
DateTimeString                  16 non-null object
LastUpdated                     16 non-null object
HasStarted                      16 non-null bool
Status                          16 non-null object
Quarter                         0 non-null float64
AwayTeamID                      16 non-null int64
AwayTeam          

In [334]:
favor_map = {}
for index, row in spread_df.iterrows():

    favor_map[row['HomeTeam']] = row['PointSpread']
    favor_map[row['AwayTeam']] = -row['PointSpread']
    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    
print(favor_map)
print('Negative means favored') # https://madduxsports.com/how-to-read-nfl-odds-lines.html

{'HOU': 7.5, 'CAR': -7.5, 'BUF': -9.5, 'WAS': 9.5, 'CLE': -7.5, 'CHI': 7.5, 'DET': 7.5, 'BAL': -7.5, 'TEN': -5.0, 'IND': 5.0, 'KC': -7.0, 'LAC': 7.0, 'NE': -2.5, 'NO': 2.5, 'NYG': -3.5, 'ATL': 3.5, 'PIT': -4.5, 'CIN': 4.5, 'JAX': 7.0, 'ARI': -7.0, 'DEN': -10.5, 'NYJ': 10.5, 'LV': -5.0, 'MIA': 5.0, 'LAR': -1.5, 'TB': 1.5, 'MIN': 1.5, 'SEA': -1.5, 'SF': -4.0, 'GB': 4.0, 'DAL': -4.0, 'PHI': 4.0, 'JAC': 7.0}
Negative means favored


In [335]:

df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())
df.to_csv(ACTIVE_FILE)
print('wrote ' + ACTIVE_FILE)
df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']
excluded_teams = set([x for x in favor_map.keys() if favor_map[x] > -MIN_FAVORED])

excluded_players = list(df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]['Name'])

excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])
if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {excluded_players}")
custom_adds = []#["Buffalo Bills"]# ["Rob Gronkowski"] #["Kyler Murray"] # [] #["Jake Kumerow"] 
if excluded_players:
    excluded_players.extend(custom_adds)


             FPPG      Played       Salary  Tier  Unnamed: 14  Unnamed: 15
count  839.000000  839.000000   839.000000   0.0          0.0          0.0
mean     4.067134    2.908224  4911.918951   NaN          NaN          NaN
std      5.311582    3.130406   969.794906   NaN          NaN          NaN
min     -0.750000    1.000000  3000.000000   NaN          NaN          NaN
25%      0.541667    2.000000  4500.000000   NaN          NaN          NaN
50%      2.283333    2.000000  4500.000000   NaN          NaN          NaN
75%      4.975000    2.000000  5300.000000   NaN          NaN          NaN
max     34.830002   18.000000  9700.000000   NaN          NaN          NaN
wrote ./data/active.csv
Excluding: ["D'Andre Swift", 'Derek Carr', 'Josh Jacobs', 'Diontae Johnson', 'Carson Wentz', 'Baker Mayfield', 'Tua Tagovailoa', 'Odell Beckham Jr.', 'Jarvis Landry', 'Ryan Fitzpatrick', 'Andy Dalton', 'Darrell Henderson Jr.', 'Marcus Mariota', 'AJ McCarron', 'Nate Stanley', 'Sam Ehlinger', 'Josh Dob

In [336]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 1],
        ['D', 1, 1]
    ]               

In [337]:
# Use salary data from csv as optimization basis.
players = salary_download.generate_players_from_csvs(
  salary_file_location=ACTIVE_FILE,
  game=rules.FAN_DUEL,
)

# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
def block_function(p):
#     if p.pos == 'D':
#         print(p.__dict__)
    return p.cost < 5000 or p.proj < 14 or favor_map[p.team] > -MIN_FAVORED or float(p.kv_store['Played']) > 10

opt_settings = OptimizerSettings(
    custom_rules=[
        CustomRule(
            group_a=lambda p: p,
            group_b=block_function, 
            comparison=lambda sum, a, b: sum(b) <= 1
        ),   
    ],
    min_teams=8
)

player_settings = PlayerPoolSettings()

if excluded_players:
    constraints = LineupConstraints(banned=[excluded_players])
else:
    constraints = LineupConstraints()

ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.defensive_positions = ['D']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

roster = run(
    rule_set=ACTIVE_RULE_SET,
    player_pool=players,
    verbose=True,
    optimizer_settings=opt_settings,
    constraints=constraints,
    player_settings=player_settings
)

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 1], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 0, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX'], 'defensive_positions': ['D'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}
Optimal roster for: NFL
+----------+--------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player             | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+--------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Daniel Jones       | NYG  | ATL@NYG |  7,400 | 25.420000076293945 |     0.00 |        |
| RB       | Derrick Henry      | TEN  | IND@TEN |  9,700 |  26.95000076293945 |     0.00 |        |
| RB       | Nick Chubb         | CLE  | CHI@CLE |  7,800 | 18.700

In [338]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:
#         return matches.iloc[0]['Id']
#     return None
roster.players[0].__dict__

{'pos': 'RB',
 'name': 'Derrick Henry',
 'cost': 9700.0,
 'team': 'TEN',
 'matchup': 'IND@TEN',
 'proj': 26.95000076293945,
 'average_score': 26.95000076293945,
 'projected_ownership_pct': 0,
 'lineup_count': 0,
 'marked': None,
 'lock': False,
 'position_lock': False,
 'ban': False,
 'position_ban': False,
 'multi_position': False,
 'possible_positions': 'RB',
 'kv_store': OrderedDict([('', '0'),
              ('Id', '64292-39280'),
              ('Position', 'RB'),
              ('First Name', 'Derrick'),
              ('Nickname', 'Derrick Henry'),
              ('Last Name', 'Henry'),
              ('FPPG', '26.95000076293945'),
              ('Played', '2.0'),
              ('Salary', '9700'),
              ('Game', 'IND@TEN'),
              ('Team', 'TEN'),
              ('Opponent', 'IND'),
              ('Injury Indicator', ''),
              ('Injury Details', ''),
              ('Tier', ''),
              ('Unnamed: 14', ''),
              ('Unnamed: 15', ''),
              (

In [339]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])

TEN Derrick Henry 26.95000076293945 26.95000076293945 9700.0   2.0
CLE Nick Chubb 18.700000762939453 18.700000762939453 7800.0   2.0
NYG Daniel Jones 25.420000076293945 25.420000076293945 7400.0   2.0
LV Darren Waller 15.25 15.25 7400.0   2.0
BAL Marquise Brown 18.350000381469727 18.350000381469727 6600.0   2.0
NYG Sterling Shepard 16.899999618530273 16.899999618530273 6400.0   2.0
ARI Rondale Moore 14.850000381469727 14.850000381469727 5600.0   2.0
BUF Zack Moss 14.399999618530273 14.399999618530273 5400.0   1.0
KC Kansas City Chiefs 6.0 6.0 3700.0   2.0


In [340]:
headers = []
players = []

for r in roster.players:
    headers.append(r.pos if r.pos != 'D' else 'DEF')
    p = f"{r.kv_store['Id']}:{r.name}"
    players.append(p)

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [341]:
output = pd.read_csv('upload.csv')
output

,RB,RB.1,QB,TE,WR,WR.1,WR.2,RB.2,DEF
0,64292-39280:Derrick Henry,64292-53562:Nick Chubb,64292-63519:Daniel Jones,64292-25011:Darren Waller,64292-86687:Marquise Brown,64292-27477:Sterling Shepard,64292-93408:Rondale Moore,64292-70027:Zack Moss,64292-12536:Kansas City Chiefs


In [342]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score

{'D': 6.076923076923077,
 'QB': 8.664906365590344,
 'RB': 4.69703548677165,
 'TE': 3.4282238323717182,
 'WR': 5.04846522460301,
 'WR/RB': 14.550000190734865,
 'DEF': 6.076923076923077}

In [343]:
expected_score = sum([m_score[h] for h in headers])
expected_score

47.40655540900913

In [344]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
25,64292-91598,RB,D'Andre,D'Andre Swift,Swift,20.400000,1.0,7400,BAL@DET,DET,BAL,Q,Groin,NaN,NaN,NaN,RB/FLEX,D'Andre Swift,0.002757
29,64292-45859,QB,Derek,Derek Carr,Carr,24.590000,2.0,7200,MIA@LV,LV,MIA,Q,Ankle,NaN,NaN,NaN,QB,Derek Carr,0.003415
31,64292-72731,RB,Josh,Josh Jacobs,Jacobs,16.500000,1.0,7200,MIA@LV,LV,MIA,Q,Toe,NaN,NaN,NaN,RB/FLEX,Josh Jacobs,0.002292
37,64292-66008,WR,Diontae,Diontae Johnson,Johnson,13.550000,2.0,7000,CIN@PIT,PIT,CIN,Q,Knee,NaN,NaN,NaN,WR/FLEX,Diontae Johnson,0.001936
38,64292-70324,QB,Carson,Carson Wentz,Wentz,18.459999,2.0,7000,IND@TEN,IND,TEN,Q,Ankle,NaN,NaN,NaN,QB,Carson Wentz,0.002637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,64292-95999,TE,Charlie,Charlie Taumoepeau,Taumoepeau,0.000000,1.0,4000,BAL@DET,DET,BAL,IR,Neck,NaN,NaN,NaN,TE/FLEX,Charlie Taumoepeau,0.000000
810,64292-40746,TE,Troy,Troy Fumagalli,Fumagalli,2.571429,7.0,4000,NO@NE,NE,NO,IR,Undisclosed,NaN,NaN,NaN,TE/FLEX,Troy Fumagalli,0.000643
815,64292-28570,TE,Nick,Nick Vannett,Vannett,2.045455,11.0,4000,NO@NE,NO,NE,IR,Knee,NaN,NaN,NaN,TE/FLEX,Nick Vannett,0.000511
818,64292-65492,TE,Dax,Dax Raymond,Raymond,0.000000,1.0,4000,CIN@PIT,PIT,CIN,Q,Undisclosed,NaN,NaN,NaN,TE/FLEX,Dax Raymond,0.000000


In [345]:
df[df['First Name'] == 'Jeff']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
390,64292-41319,WR,Jeff,Jeff Badet,Badet,0.0,2.0,4500,ATL@NYG,ATL,NYG,Q,Lower leg,NaN,NaN,NaN,WR/FLEX,Jeff Badet,0.000000
457,64292-64350,WR,Jeff,Jeff Smith,Smith,2.0,2.0,4500,NYJ@DEN,NYJ,DEN,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Jeff Smith,0.000444
544,64292-130343,WR,Jeff,Jeff Cotton Jr.,Cotton Jr.,0.0,1.0,4500,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Jeff Cotton Jr.,0.000000


In [346]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
159,64292-30733,WR/RB,Cordarrelle,Cordarrelle Patterson,Patterson,14.55,2.0,5900,ATL@NYG,ATL,NYG,NaN,NaN,NaN,NaN,NaN,RB/WR/FLEX,Cordarrelle Patterson,0.002466


In [347]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

,FPPG,Nickname,Salary,Salary/FPPG
691,12.5,Buffalo Bills,4100,0.003049
826,10.5,Tampa Bay Buccaneers,3900,0.002692
282,12.0,New England Patriots,4700,0.002553
677,10.5,New Orleans Saints,4300,0.002442
833,8.5,Minnesota Vikings,3500,0.002429
692,9.5,Chicago Bears,4100,0.002317
713,8.0,Los Angeles Rams,4000,0.002000
827,7.5,Indianapolis Colts,3800,0.001974
312,8.5,Arizona Cardinals,4600,0.001848
231,8.5,Denver Broncos,5000,0.001700


In [348]:
df[df['Team'] == 'JAC']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
57,64292-89951,QB,Trevor,Trevor Lawrence,Lawrence,15.450000,2.0,6600,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,QB,Trevor Lawrence,0.002341
77,64292-22103,WR,Marvin,Marvin Jones Jr.,Jones Jr.,15.350000,2.0,6300,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Marvin Jones Jr.,0.002437
121,64292-83079,QB,Kyle,Kyle Lauletta,Lauletta,2.283333,2.0,6000,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,QB,Kyle Lauletta,0.000381
129,64292-27986,QB,C.J.,C.J. Beathard,Beathard,7.468571,7.0,6000,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,QB,C.J. Beathard,0.001245
167,64292-52917,WR,DJ,DJ Chark Jr.,Chark Jr.,9.250000,2.0,5800,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,WR/FLEX,DJ Chark Jr.,0.001595
192,64292-130288,RB,James,James Robinson,Robinson,7.400000,2.0,5500,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,RB/FLEX,James Robinson,0.001345
204,64292-87859,WR,Laviska,Laviska Shenault Jr.,Shenault Jr.,5.050000,2.0,5400,ARI@JAC,JAC,ARI,Q,Shoulder,NaN,NaN,NaN,WR/FLEX,Laviska Shenault Jr.,0.000935
228,64292-28473,RB,Carlos,Carlos Hyde,Hyde,3.750000,2.0,5100,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,RB/FLEX,Carlos Hyde,0.000735
292,64292-62508,TE,James,James O'Shaughnessy,O'Shaughnessy,5.350000,2.0,4700,ARI@JAC,JAC,ARI,O,Ankle,NaN,NaN,NaN,TE/FLEX,James O'Shaughnessy,0.001138
324,64292-60705,WR,Tyron,Tyron Johnson,Johnson,0.550000,2.0,4600,ARI@JAC,JAC,ARI,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Tyron Johnson,0.000120


In [349]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

HomeTeam       BUF
PointSpread   -9.5
Name: 1, dtype: object